In [3]:
# =========================================
# 1. IMPORTAÇÃO DE BIBLIOTECAS
# =========================================
import pandas as pd
import numpy as np

from IPython.display import display

# Modelagem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# =========================================
# 2. LEITURA DOS DADOS
# =========================================
# Exemplo de leitura (substitua pelos seus caminhos reais)
orders = pd.read_csv("orders.csv")
drivers = pd.read_csv("drivers_data.csv")
customers = pd.read_csv("customers_data.csv")
products = pd.read_csv("products_data.csv")
missing_items = pd.read_csv("missing_items_data.csv")

# =========================================
# 3. TRATAMENTO DA COLUNA 'order_amount'
# =========================================
# Remove todos os caracteres que não sejam dígitos, ponto, vírgula ou sinal de menos
# Assim garantimos que '$' e outros símbolos sejam removidos.
orders["order_amount"] = orders["order_amount"].replace(r"[^\d.,-]", "", regex=True)

# Se houver vírgula como separador decimal, converta para ponto
orders["order_amount"] = orders["order_amount"].str.replace(",", ".")

# Converta para número, forçando valores inválidos para NaN
orders["order_amount"] = pd.to_numeric(orders["order_amount"], errors="coerce")

# Opcional: eliminar linhas que ficaram com NaN após a conversão
orders.dropna(subset=["order_amount"], inplace=True)

# Verificar rapidamente se a conversão deu certo
print("Tipo de 'order_amount':", orders["order_amount"].dtype)
print("Alguns valores de 'order_amount':")
display(orders["order_amount"].head())


# Corrigir nome da coluna em products
products.rename(columns={"produc_id": "product_id"}, inplace=True)

# =========================================
# 4. MERGES E JUNÇÕES
# =========================================
# Exemplo de merge de todos os DataFrames em um só (ajuste conforme sua necessidade)

# Merge com drivers
orders_drivers = orders.merge(drivers, on="driver_id", how="left")

# Merge com customers
orders_drivers_customers = orders_drivers.merge(customers, on="customer_id", how="left")

# Merge com products (se necessário)
orders_products = orders_drivers_customers.merge(products, on="product_id", how="left")

# Merge com missing_items (se necessário, usando múltiplas chaves se existirem)
walmart_join = orders_products.merge(
    missing_items, 
    on=["order_id", "product_id"], 
    how="left"
)

# Exemplo de checagem rápida
print("Dimensões de walmart_join:", walmart_join.shape)
display(walmart_join.head())


# =========================================
# 5. EVENTUAL ANÁLISE/EXPLORAÇÃO
# =========================================
# Exemplo de estatísticas simples
print("\nEstatísticas de 'order_amount':")
display(walmart_join["order_amount"].describe())

# Exemplo de distribuição de classes
# (Se houver uma coluna 'fraude_tipo', por exemplo)
if "fraude_tipo" in walmart_join.columns:
    class_distribution = walmart_join["fraude_tipo"].value_counts()
    print("\nDistribuição da Variável Alvo (Fraude Tipo):")
    display(class_distribution.to_frame())


# =========================================
# 6. PREPARAÇÃO PARA MODELO
# =========================================
# Exemplo: se quisermos prever 'fraude_tipo', se ela for a coluna alvo
if "fraude_tipo" in walmart_join.columns:
    # Defina as colunas de features (ajuste conforme seu caso)
    features = [
        "order_amount", 
        # ... inclua outras colunas relevantes ...
    ]
    target = "fraude_tipo"

    # Filtra apenas linhas sem valores nulos em features e target
    walmart_join_model = walmart_join.dropna(subset=features + [target])

    X = walmart_join_model[features]
    y = walmart_join_model[target]

    # Exemplo de train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # =========================================
    # 7. TREINAMENTO DE MODELO (Random Forest)
    # =========================================
    # Ajuste os pesos das classes, se necessário
    # Exemplo: class_weight = dict(...)
    # class_weights = {0: 1.0, 1: 3.0}  # Exemplo de peso para classe minoritária
    class_weights = "balanced"  # ou "balanced_subsample", ou um dicionário

    rf_model = RandomForestClassifier(
        n_estimators=100,
        class_weight=class_weights,
        random_state=42
    )

    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)

    # =========================================
    # 8. AVALIAÇÃO DO MODELO
    # =========================================
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))

    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

else:
    print("\nNão foi encontrada a coluna 'fraude_tipo' para modelagem.")


Tipo de 'order_amount': float64
Alguns valores de 'order_amount':


1    659.11
2    251.45
3    598.83
4     27.18
5    494.32
Name: order_amount, dtype: float64

KeyError: 'product_id'